# MODIS EVI - Training second set of models

## Importing Cleaned Data

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/  # change this to the root working directory

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
import pandas as pd

In [ ]:
file_path = 'Modeling VIs in Tallgrass/data/P13_Cleaned_Data.csv'
df = pd.read_csv(file_path)
df

,Unnamed: 0,DATE,Tavg_mean,Havg_mean,Vdef_mean,Hdeg_mean,Cdeg_mean,Wspd_mean,Atot_mean,Rain_sum,Savg_mean,Bavg_mean,Tr05_mean,Tr25_mean,Tr60_mean,MODIS_EVI,LSWI
0,6,2000-02-18,52.10875,71.93625,4.46000,12.16500,0.00000,14.49375,13.258750,1.06,50.37625,50.81500,1.863600,1.581400,1.504100,0.254,-0.111
1,7,2000-02-26,48.71250,66.61750,5.25500,16.66875,0.00000,12.51125,14.832385,0.78,49.21875,49.43125,1.671738,1.570663,1.491400,0.211,-0.136
2,8,2000-03-05,50.03750,72.90625,4.11375,14.25750,0.00000,13.49750,17.770000,0.71,51.34375,51.30125,1.562450,1.557400,1.478662,0.231,-0.092
3,9,2000-03-13,47.88250,81.58250,3.14125,15.82625,0.00000,10.42750,12.791250,0.95,50.05750,49.97750,1.594888,1.547525,1.471650,0.170,-0.205
4,10,2000-03-21,57.95000,75.16625,5.15500,6.97000,0.00000,10.11750,16.667500,2.39,56.53125,56.36875,1.507513,1.531750,1.465475,0.187,-0.206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,1090,2023-09-14,71.42375,73.00250,8.62500,0.00000,6.97375,8.59875,15.427500,0.99,71.31000,71.28000,1.520763,3.208450,3.473537,0.379,0.084
1070,1091,2023-09-22,75.57250,66.09000,12.62500,0.00000,11.76625,9.57000,18.105000,0.39,72.30125,75.01625,1.620875,2.699000,3.452688,0.446,0.163
1071,1092,2023-09-30,68.66625,63.53250,10.45000,2.35625,7.10750,8.96625,15.098750,1.52,69.67000,72.41000,1.813212,2.058113,3.441438,0.426,0.125
1072,1093,2023-10-08,61.79500,65.17125,7.83500,4.42000,1.91250,12.27000,15.991250,0.00,64.46500,64.92125,1.711912,1.664287,3.413925,0.401,0.105


In [ ]:
df = df.drop(['Unnamed: 0'], axis = 1)

## Train and Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Training data from 2000-2021 and testing data from 2022-2023
train_test = df[df['DATE'] < '2022-01-01']
validation = df[df['DATE'] >= '2022-01-01']

In [ ]:
X = train_test[['Tavg_mean', 'Havg_mean', 'Vdef_mean', 'Hdeg_mean', 'Cdeg_mean',
       'Wspd_mean', 'Atot_mean', 'Rain_sum', 'Savg_mean', 'Bavg_mean',
       'Tr05_mean', 'Tr25_mean', 'Tr60_mean']]
y = train_test[['MODIS_EVI']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_val = validation[['Tavg_mean', 'Havg_mean', 'Vdef_mean', 'Hdeg_mean', 'Cdeg_mean',
       'Wspd_mean', 'Atot_mean', 'Rain_sum', 'Savg_mean', 'Bavg_mean',
       'Tr05_mean', 'Tr25_mean', 'Tr60_mean']]
y_val = validation[['MODIS_EVI']]

In [ ]:
len(X_train), len(X_test), len(X_val)

(802, 201, 71)

In [ ]:
# Filter observations where EVI < -0.07
filtered_indices = y_test[y_test['MODIS_EVI'] < -0.07].index

# Print the indices of filtered observations
print(filtered_indices)

# Print the specific observations
print(y.loc[filtered_indices])

Index([456], dtype='int64')
     MODIS_EVI
456     -0.085


In [ ]:
# Remove observation with index 456 from X_test and y_test
X_test = X_test.drop(index=456)
y_test = y_test.drop(index=456)

In [ ]:
len(X_train), len(X_test), len(X_val)

(802, 200, 71)

## Training More Models

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Import packages
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ElasticNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import LSTM
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
import numpy as np

### Bayesian Ridge

In [ ]:
# Ensure y_train and y_test are 1D arrays
y_train = y_train.to_numpy().flatten()
y_test = y_test.to_numpy().flatten()

# Train a Bayesian Ridge regression model
bayesian_ridge_model = BayesianRidge()
bayesian_ridge_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred_br = bayesian_ridge_model.predict(X_test)

# Calculate the MSE, MAE, RMSE, and R² score on the testing set
mse_br_test = mean_squared_error(y_test, y_pred_br)
mae_br_test = mean_absolute_error(y_test, y_pred_br)
rmse_br_test = mean_squared_error(y_test, y_pred_br, squared=False)
r2_br_test = r2_score(y_test, y_pred_br)

print('MSE:', mse_br_test)
print('MAE:', mae_br_test)
print('RMSE:', rmse_br_test)
print('R2 score:', r2_br_test)

MSE: 0.004103790440592999
MAE: 0.0511631998962259
RMSE: 0.06406083390491416
R2 score: 0.7652426671878233


In [ ]:
# Tuned
# Define parameter grid

# Define the model
bayesian_ridge_model = BayesianRidge()

# Define the parameter grid
param_grid = {
    'max_iter': [100, 200, 300],
    'alpha_1': [1e-6, 1e-5, 1e-4],
    'alpha_2': [1e-5, 1e-4, 1e-3],
    'lambda_1': [1e-5, 1e-4, 1e-3],
    'lambda_2': [1e-5, 1e-4, 1e-3],
    'tol': [1e-4, 1e-3, 1e-2]
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=bayesian_ridge_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the model using grid search
grid_search.fit(X_train, y_train)

# Get the best model from grid search
tuned_br_best_model = grid_search.best_estimator_

# Make predictions on the testing set
y_pred_br_tuned = tuned_br_best_model.predict(X_test)

# Calculate the MSE, MAE, RMSE, and R² score on the testing set
mse_br_tuned_test = mean_squared_error(y_test, y_pred_br_tuned)
mae_br_tuned_test = mean_absolute_error(y_test, y_pred_br_tuned)
rmse_br_tuned_test = mean_squared_error(y_test, y_pred_br_tuned, squared=False)
r2_br_tuned_test = r2_score(y_test, y_pred_br_tuned)

print('Best Parameters:', grid_search.best_params_)
print('MSE:', mse_br_tuned_test)
print('MAE:', mae_br_tuned_test)
print('RMSE:', rmse_br_tuned_test)
print('R2 score:', r2_br_tuned_test)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best Parameters: {'alpha_1': 1e-06, 'alpha_2': 0.001, 'lambda_1': 0.001, 'lambda_2': 0.0001, 'max_iter': 100, 'tol': 0.0001}
MSE: 0.004099940273949911
MAE: 0.05115556256810553
RMSE: 0.06403077599053374
R2 score: 0.7654629159712586


### Elastic Net

In [ ]:
# Define the Elastic Net model
elastic_net_model = ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42)

# Train the model
elastic_net_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred_en = elastic_net_model.predict(X_test)

# Calculate the MSE, MAE, RMSE, and R² score on the testing set
mse_en_test = mean_squared_error(y_test, y_pred_en)
mae_en_test = mean_absolute_error(y_test, y_pred_en)
rmse_en_test = mean_squared_error(y_test, y_pred_en, squared=False)
r2_en_test = r2_score(y_test, y_pred_en)

print('MSE:', mse_en_test)
print('MAE:', mae_en_test)
print('RMSE:', rmse_en_test)
print('R2 score:', r2_en_test)

MSE: 0.006927000712012644
MAE: 0.0676187829752003
RMSE: 0.08322860513076405
R2 score: 0.603740923158553


In [ ]:
# Tuned
# Define parameter grid

# Define the parameter grid
param_grid = {
    'alpha': [0.1, 1.0, 10.0, 100.0],  # Regularization strength
    'l1_ratio': [0.1, 0.5, 0.7, 0.9, 1.0]  # Mix between L1 and L2 penalty
}

# Set up GridSearchCV
grid_search_en = GridSearchCV(estimator=ElasticNet(random_state=42),
                              param_grid=param_grid,
                              cv=5,  # 5-fold cross-validation
                              scoring='neg_mean_squared_error',
                              n_jobs=-1)  # Use all available cores

# Fit the model using grid search
grid_search_en.fit(X_train, y_train)

# Get the best model from grid search
best_en_model = grid_search_en.best_estimator_

# Make predictions on the testing set
y_pred_en_tuned = best_en_model.predict(X_test)

# Calculate the MSE, MAE, RMSE, and R² score on the testing set
mse_en_tuned_test = mean_squared_error(y_test, y_pred_en_tuned)
mae_en_tuned_test = mean_absolute_error(y_test, y_pred_en_tuned)
rmse_en_tuned_test = mean_squared_error(y_test, y_pred_en_tuned, squared=False)
r2_en_tuned_test = r2_score(y_test, y_pred_en_tuned)

print('Best Parameters:', grid_search_en.best_params_)
print('MSE:', mse_en_tuned_test)
print('MAE:', mae_en_tuned_test)
print('RMSE:', rmse_en_tuned_test)
print('R2 score:', r2_en_tuned_test)

Best Parameters: {'alpha': 0.1, 'l1_ratio': 0.1}
MSE: 0.0043147464440773465
MAE: 0.052412804820090114
RMSE: 0.06568672958883968
R2 score: 0.7531749289746739


### ANN

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Define the neural network architecture
ann_model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer with 1 neuron for EVI prediction
])

# Compile the model
ann_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
ann_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = ann_model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Make predictions
predictions = ann_model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0592 - val_loss: 0.0183
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0163 - val_loss: 0.0121
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0102 - val_loss: 0.0117
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0090 - val_loss: 0.0094
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0076 - val_loss: 0.0091
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0065 - val_loss: 0.0086
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0067 - val_loss: 0.0096
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0056 - val_loss: 0.0087
Epoch 9/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0057 - val_loss: 0.0077
Epoch 10/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0051 - val_loss: 0.0075
Epoch 11/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0045 - val_loss: 0.0076
Epoch 12/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0046 - val_l

In [ ]:
# Calculate evaluation metrics
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)
print('MSE:', mse)
print('RMSE:', rmse)
print('R2 score:', r2)
print('MAE:', mae)

MSE: 0.004388142585388876
RMSE: 0.06624305688439261
R2 score: 0.7489763026991786
MAE: 0.05081269991874695


In [ ]:
# Tuned
# Define parameter grid
# Could not do parameter grid because KerasRegressor could not be imported for grid search

# Define the neural network architecture
tuned_ann_model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer with 1 neuron for EVI prediction
])

# Compile the model
tuned_ann_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
tuned_ann_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = tuned_ann_model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Make predictions
y_pred = tuned_ann_model.predict(X_test)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0631 - val_loss: 0.0196
Epoch 2/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0183 - val_loss: 0.0129
Epoch 3/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0103 - val_loss: 0.0105
Epoch 4/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0085 - val_loss: 0.0089
Epoch 5/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0077 - val_loss: 0.0079
Epoch 6/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0063 - val_loss: 0.0071
Epoch 7/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0065 - val_loss: 0.0066
Epoch 8/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0049 - val_loss: 0.0061
Epoch 9/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0047 - val_loss: 0.0060
Epoch 10/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0044 - val_loss: 0.0055
Epoch 11/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0047 - val_loss: 0.0065
Epoch 12/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0045 - val_l

In [ ]:
# Calculate regression metrics
mse_test = mean_squared_error(y_test, y_pred)
mae_test = mean_absolute_error(y_test, y_pred)
rmse_test = mean_squared_error(y_test, y_pred, squared=False)
r2_test = r2_score(y_test, y_pred)

# Remember manual best parameters used
print('Test MSE:', mse_test)
print('Test MAE:', mae_test)
print('Test RMSE:', rmse_test)
print('Test R2 score:', r2_test)

Test MSE: 0.0061363106305693535
Test MAE: 0.059282654881477354
Test RMSE: 0.07833460685143798
Test R2 score: 0.6489723493943058


### CNN

In [ ]:
# Define the CNN model
cnn_model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer with 1 neuron for EVI prediction
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
cnn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = cnn_model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Make predictions
predictions = cnn_model.predict(X_test)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0979 - val_loss: 0.0135
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0122 - val_loss: 0.0092
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0072 - val_loss: 0.0063
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0088 - val_loss: 0.0058
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0063 - val_loss: 0.0053
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0049 - val_loss: 0.0057
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0054 - val_loss: 0.0055
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0044 - val_loss: 0.0053
Epoch 9/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0050 - val_loss: 0.0051
Epoch 10/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0053 - val_loss: 0.0053
Epoch 11/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0049 - val_loss: 0.0062
Epoch 12/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0063 - val_loss: 0.0054

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [ ]:
# Calculate evaluation metrics
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)
print('MSE:', mse)
print('RMSE:', rmse)
print('R2 score:', r2)
print('MAE:', mae)

MSE: 0.003977243141903594
RMSE: 0.0630653878280598
R2 score: 0.772481805429643
MAE: 0.048462953539192666


In [ ]:
# Tuned
# Define parameter grid

# Define the CNN model
tuned_cnn_model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer with 1 neuron for EVI prediction
])

# Compile the model
tuned_cnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
tuned_cnn_model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = tuned_cnn_model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Make predictions
y_pred = tuned_cnn_model.predict(X_test)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0550 - val_loss: 0.0075
Epoch 2/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0090 - val_loss: 0.0066
Epoch 3/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0065 - val_loss: 0.0055
Epoch 4/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0056 - val_loss: 0.0068
Epoch 5/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0055 - val_loss: 0.0055
Epoch 6/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0047 - val_loss: 0.0058
Epoch 7/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0045 - val_loss: 0.0054
Epoch 8/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0044 - val_loss: 0.0053
Epoch 9/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0050 - val_loss: 0.0056
Epoch 10/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0047 - val_loss: 0.0060
Epoch 11/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0052 - val_loss: 0.0064
Epoch 12/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0078 - val_loss: 0.0053

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [ ]:
# Calculate regression metrics
mse_test = mean_squared_error(y_test, y_pred)
mae_test = mean_absolute_error(y_test, y_pred)
rmse_test = mean_squared_error(y_test, y_pred, squared=False)
r2_test = r2_score(y_test, y_pred)

print('Test MSE:', mse_test)
print('Test MAE:', mae_test)
print('Test RMSE:', rmse_test)
print('Test R2 score:', r2_test)

Test MSE: 0.004315461073983803
Test MAE: 0.05125590820252895
Test RMSE: 0.06569216904611845
Test R2 score: 0.7531340485707604


### RNN

In [ ]:
# Reshape input data for RNN
X_train_rnn = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_rnn = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [ ]:
# Define the RNN model
rnn_model = Sequential([
    SimpleRNN(64, activation='relu', input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True),
    SimpleRNN(32, activation='relu'),
    Dense(1)  # Output layer with 1 neuron for EVI prediction
])

# Compile the model
rnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
rnn_model.fit(X_train_rnn, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = rnn_model.evaluate(X_test_rnn, y_test)
print("Test Loss:", loss)

# Make predictions
predictions = rnn_model.predict(X_test_rnn)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.2504 - val_loss: 0.0426
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0306 - val_loss: 0.0211
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0135 - val_loss: 0.0167
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0109 - val_loss: 0.0134
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0088 - val_loss: 0.0126
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0071 - val_loss: 0.0111
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0073 - val_loss: 0.0114
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0065 - val_loss: 0.0107
Epoch 9/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0053 - val_loss: 0.0100
Epoch 10/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0070 - val_loss: 0.0098
Epoch 11/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0055 - val_loss: 0.0110
Epoch 12/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0062 - val_loss: 0.0093

In [ ]:
# Calculate evaluation metrics
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)
print('MSE:', mse)
print('RMSE:', rmse)
print('R2 score:', r2)
print('MAE:', mae)

MSE: 0.0054067566208485345
RMSE: 0.07353065089368198
R2 score: 0.6907064866373712
MAE: 0.05827935701012612


In [ ]:
# Tuned
# Define parameter grid

# Define the RNN model
tuned_rnn_model = Sequential([
    SimpleRNN(64, activation='relu', input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True),
    SimpleRNN(32, activation='relu'),
    Dense(1)  # Output layer with 1 neuron for EVI prediction
])

# Compile the model
tuned_rnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
tuned_rnn_model.fit(X_train_rnn, y_train, epochs=110, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = tuned_rnn_model.evaluate(X_test_rnn, y_test)
print("Test Loss:", loss)

# Make predictions
y_pred = tuned_rnn_model.predict(X_test_rnn)

Epoch 1/110


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 0.0924 - val_loss: 0.0397
Epoch 2/110
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0302 - val_loss: 0.0204
Epoch 3/110
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0157 - val_loss: 0.0138
Epoch 4/110
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0117 - val_loss: 0.0120
Epoch 5/110
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0084 - val_loss: 0.0108
Epoch 6/110
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0092 - val_loss: 0.0100
Epoch 7/110
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0075 - val_loss: 0.0100
Epoch 8/110
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0073 - val_loss: 0.0101
Epoch 9/110
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0059 - val_loss: 0.0091
Epoch 10/110
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0054 - val_loss: 0.0086
Epoch 11/110
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0050 - val_loss: 0.0085
Epoch 12/110
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0053 - val_l

In [ ]:
# Calculate regression metrics
mse_test = mean_squared_error(y_test, y_pred)
mae_test = mean_absolute_error(y_test, y_pred)
rmse_test = mean_squared_error(y_test, y_pred, squared=False)
r2_test = r2_score(y_test, y_pred)

print('Test MSE:', mse_test)
print('Test MAE:', mae_test)
print('Test RMSE:', rmse_test)
print('Test R2 score:', r2_test)

Test MSE: 0.005326798284126751
Test MAE: 0.0553848849710822
Test RMSE: 0.07298491819634212
Test R2 score: 0.6952805033023624


### LSTM

In [ ]:
# Define the LSTM model
timesteps=1
X_train_lstm = X_train.reshape(X_train.shape[0], timesteps, X_train.shape[1])
X_test_lstm = X_test.reshape(X_test.shape[0], timesteps, X_test.shape[1])

lstm_model = Sequential([
    LSTM(64, activation='relu', input_shape=(timesteps, X_train_lstm.shape[2]), return_sequences=True),
    LSTM(32, activation='relu'),
    Dense(1)  # Output layer with 1 neuron for GPP prediction
])

# Compile the model
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = lstm_model.evaluate(X_test_lstm, y_test)
print("Test Loss:", loss)

# Make predictions
predictions = lstm_model.predict(X_test_lstm)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 0.1032 - val_loss: 0.0743
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0492 - val_loss: 0.0225
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0165 - val_loss: 0.0086
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0090 - val_loss: 0.0070
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0059 - val_loss: 0.0057
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0061 - val_loss: 0.0049
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0048 - val_loss: 0.0046
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0045 - val_loss: 0.0044
Epoch 9/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0043 - val_loss: 0.0043
Epoch 10/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0045 - val_loss: 0.0042
Epoch 11/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0039 - val_loss: 0.0043
Epoch 12/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0042 - val_loss: 0.0042

In [ ]:
# Calculate evaluation metrics
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)
print('MSE:', mse)
print('RMSE:', rmse)
print('R2 score:', r2)
print('MAE:', mae)

MSE: 0.00319772667077756
RMSE: 0.05654844534359508
R2 score: 0.8170740452854054
MAE: 0.04299417999446392


In [ ]:
# Tuned
# Define parameter grid

tuned_lstm_model = Sequential([
    LSTM(64, activation='relu', input_shape=(timesteps, X_train_lstm.shape[2]), return_sequences=True),
    LSTM(32, activation='relu'),
    Dense(1)  # Output layer with 1 neuron for GPP prediction
])

# Compile the model
tuned_lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
tuned_lstm_model.fit(X_train_lstm, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = tuned_lstm_model.evaluate(X_test_lstm, y_test)
print("Test Loss:", loss)

# Make predictions
y_pred = tuned_lstm_model.predict(X_test_lstm)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - loss: 0.0938 - val_loss: 0.0593
Epoch 2/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0424 - val_loss: 0.0176
Epoch 3/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0151 - val_loss: 0.0068
Epoch 4/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0068 - val_loss: 0.0055
Epoch 5/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0050 - val_loss: 0.0051
Epoch 6/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0053 - val_loss: 0.0048
Epoch 7/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0050 - val_loss: 0.0046
Epoch 8/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0040 - val_loss: 0.0045
Epoch 9/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0042 - val_loss: 0.0044
Epoch 10/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0041 - val_loss: 0.0043
Epoch 11/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0042 - val_loss: 0.0043
Epoch 12/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0039 - val_l

In [ ]:
# Calculate regression metrics
mse_test = mean_squared_error(y_test, y_pred)
mae_test = mean_absolute_error(y_test, y_pred)
rmse_test = mean_squared_error(y_test, y_pred, squared=False)
r2_test = r2_score(y_test, y_pred)

print('Test MSE:', mse_test)
print('Test MAE:', mae_test)
print('Test RMSE:', rmse_test)
print('Test R2 score:', r2_test)

Test MSE: 0.0033061862340365237
Test MAE: 0.04336923230230809
Test RMSE: 0.05749944551068752
Test R2 score: 0.8108696159517845


## Saving Models

In [ ]:
import pickle

# Save non-tuned and tuned models
models_base_path = 'Modeling VIs in Tallgrass/models/evi/'

with open(models_base_path + 'bayesian_ridge_evi.pkl', 'wb') as file:
    pickle.dump(bayesian_ridge_model, file)

with open(models_base_path + 'tuned_br_evi.pkl', 'wb') as file:
    pickle.dump(tuned_br_best_model, file)

with open(models_base_path + 'elastic_net_evi.pkl', 'wb') as file:
    pickle.dump(elastic_net_model, file)

with open(models_base_path + 'tuned_en_evi.pkl', 'wb') as file:
    pickle.dump(best_en_model, file)

with open(models_base_path + 'ann_evi.pkl', 'wb') as file:
    pickle.dump(ann_model, file)

with open(models_base_path + 'tuned_ann_evi.pkl', 'wb') as file:
    pickle.dump(tuned_ann_model, file)

with open(models_base_path + 'cnn_evi.pkl', 'wb') as file:
    pickle.dump(cnn_model, file)

with open(models_base_path + 'tuned_cnn_evi.pkl', 'wb') as file:
    pickle.dump(tuned_cnn_model, file)

with open(models_base_path + 'rnn_evi.pkl', 'wb') as file:
    pickle.dump(rnn_model, file)

with open(models_base_path + 'tuned_rnn_evi.pkl', 'wb') as file:
    pickle.dump(tuned_rnn_model, file)

with open(models_base_path + 'lstm_evi.pkl', 'wb') as file:
    pickle.dump(lstm_model, file)

with open(models_base_path + 'tuned_lstm_evi.pkl', 'wb') as file:
    pickle.dump(tuned_lstm_model, file)

In [ ]:
# End of code 8/24/24